In [1]:
import pandas as pd
import numpy as np
from functions_NLM import estimate_nested_logit

In [2]:
# DATA INPUT
datafilename = "../data/swissmetro.dat"

# read data file
data = pd.read_table(datafilename)

In [3]:
# determine number of individuals in the data
N = data['ID'].nunique()

# DEFINE MODEL PARAMETERS
beta = np.array([0, 0, 0, 0, 1, 1, 0])
beta_names = ["ASC_CAR", "ASC_SM", "BETA_TT", "BETA_TC", "lambda_CAR_TRAIN", "lambda_SM_TRAIN", "base_alpha_TRAIN_WITH_CAR"]
fixed_params = {}

# DEFINE MODEL STRUCTURE AND LIKELIHOOD FUNCTION
def log_likelihood_SM_CNLM(beta, data):
    alpha_TRAIN_WITH_CAR = np.exp(beta[6]) / (1 + np.exp(beta[6]))
    alpha_TRAIN_WITH_SM = 1 - alpha_TRAIN_WITH_CAR

    # define utility functions
    data['U_TRAIN'] = beta[2] * data['TRAIN_TT'] + beta[3] * (data['GA'] == 0) * data['TRAIN_CO']
    data['U_SM'] = beta[1] + beta[2] * data['SM_TT'] + beta[3] * (data['GA'] == 0) * data['SM_CO']
    data['U_CAR'] = beta[0] + beta[2] * data['CAR_TT'] + beta[3] * data['CAR_CO']

    # combined utility terms inside nests
    data['U_CAR_TRAIN'] = (alpha_TRAIN_WITH_CAR * data['TRAIN_AV'] * np.exp(data['U_TRAIN'])) ** (1 / beta[4]) + \
                          (1 * data['CAR_AV'] * np.exp(data['U_CAR'])) ** (1 / beta[4])
    data['U_TRAIN_SM'] = (alpha_TRAIN_WITH_SM * data['TRAIN_AV'] * np.exp(data['U_TRAIN'])) ** (1 / beta[5]) + \
                         (1 * data['SM_AV'] * np.exp(data['U_SM'])) ** (1 / beta[5])

    # Nest probabilities
    data['P_nest_CAR_TRAIN'] = (data['U_CAR_TRAIN'] ** beta[4]) / \
                                (data['U_CAR_TRAIN'] ** beta[4] + data['U_TRAIN_SM'] ** beta[5])
    data['P_nest_TRAIN_SM'] = 1 - data['P_nest_CAR_TRAIN']

    # Within nest probabilities for any nests with > 1 alt
    data['P_TRAIN_in_CAR_TRAIN'] = ((alpha_TRAIN_WITH_CAR * data['TRAIN_AV'] * np.exp(data['U_TRAIN'])) ** (1 / beta[4])) / data['U_CAR_TRAIN']
    data['P_CAR_in_CAR_TRAIN'] = 1 - data['P_TRAIN_in_CAR_TRAIN']
    data['P_TRAIN_in_TRAIN_SM'] = ((alpha_TRAIN_WITH_SM * data['TRAIN_AV'] * np.exp(data['U_TRAIN'])) ** (1 / beta[5])) / data['U_TRAIN_SM']
    data['P_SM_in_TRAIN_SM'] = 1 - data['P_TRAIN_in_TRAIN_SM']

    # Full probabilities
    data['P_TRAIN'] = data['P_nest_CAR_TRAIN'] * data['P_TRAIN_in_CAR_TRAIN'] + data['P_nest_TRAIN_SM'] * data['P_TRAIN_in_TRAIN_SM']
    data['P_SM'] = data['P_nest_TRAIN_SM'] * data['P_SM_in_TRAIN_SM']
    data['P_CAR'] = data['P_nest_CAR_TRAIN'] * data['P_CAR_in_CAR_TRAIN']

    # calculate probability for chosen alternative for each row in the data table
    data['P'] = ((data['CHOICE'] == 1) * data['P_TRAIN'] +
                 (data['CHOICE'] == 2) * data['P_SM'] +
                 (data['CHOICE'] == 3) * data['P_CAR'])

    # take product across choices for the same person (likelihood)
    # then take the log for log-likelihood
    LL = data.groupby('ID')['P'].prod().apply(np.log).sum()

    return -LL  # We minimize negative log-likelihood

In [4]:
# Estimate parameters
result, se, t_stat, p_value, aic, bic = estimate_nested_logit(data, beta, beta_names, log_likelihood_SM_CNLM)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-pack

Optimization Results:
                   Parameter  Estimate  Robust Asymptotic SE  t-statistic  \
0                    ASC_CAR -0.234010              0.000152 -1543.924428   
1                     ASC_SM -0.126667              0.031472    -4.024814   
2                    BETA_TT -0.008182              0.000285   -28.679772   
3                    BETA_TC -0.006336              0.000210   -30.177470   
4           lambda_CAR_TRAIN  0.357802              0.000323  1108.164002   
5            lambda_SM_TRAIN  0.318159              0.022249    14.299887   
6  base_alpha_TRAIN_WITH_CAR  0.173009              0.001769    97.808558   

    p-value  
0  0.000000  
1  0.000057  
2  0.000000  
3  0.000000  
4  0.000000  
5  0.000000  
6  0.000000  
AIC: 16986.019515768065
BIC: 17036.972047912157
Log-likelihood: -8486.009757884032
